## Principal Component Analysis

The basic idea behind Principal Component Analysis(PCA), is to reduce the features of data in such a way that we still have the highest amount of information from the original features. Mathematically, it is eigendecomposition of covariance matrix of the data.

When not to use PCA:

1. If the variance of data features is very less.
2. If the features have huge outliers in the data(affects variance)


Questions:

1. Why do we find eigen vectors of mean zeroed covariance matrix?

   Because we are not changing the data, the covariance remains the same but we look at the data with new orthogonal dimensions(eigen vectors of symmetric matrices are real and othrogonal).We choose these dimensions using the eigen values(which show variance of data when projected) of the eigen vectors
   
2. How do the eigen values of these eigen vectors show variance of the projection of original matrix along these vectors?

    Ans: https://math.stackexchange.com/questions/2147211/why-are-the-eigenvalues-of-a-covariance-matrix-equal-to-the-variance-of-its-eige

Note: the below implementation uses the covariance method and involves calculating covariance matrix which is not cost effective for bigger data. 

#### Algorithm:

Lets say we have the input data as $X \in R^{n\times p}$, where $n$ are the total observations, $p$ is the number of original features. We want to reduce the dimension from $p$ to $l$ such that $l<p$ and get the final matrix $W \in R^{n\times l}$

In [1]:
import torch

##### 1. Shit the data to zero mean

We find column wise mean for each feature $\mu \in R^{p \times 1}$. We get $B$ from $X$ such that $B = X - h\mu^{T}$, where, $h \in R^{n \times 1}$

##### Scaling:
Depending on your problem, you should choose to scale the values using the standard derivation or not. Why choose scaling, to have your data points be independent of the units of their measurement. When to avoid scaling, when the variation of a feature is itself important for analysis.

In [2]:
def columnwise_mean(x):
    assert len(x.shape)==2
    
    return x.mean(dim=0)

In [3]:
def shift_mean(x, mean, scale = True):
    if scale:
        return x - mean    
    else:
        return (x - mean)/(x.std(dim=0)+1e-10)

##### 2. Find Covariance matrix:
We find the covariance matrix,$C \in R^{p \times p}$ of the mean shifted matrix $B$.
\begin{equation*} C = \frac{1}{n-1}B*B\end{equation*}
where $*$ is a conjugate transpose about which I have no idea but when all the numbers in $B$ are real the operation is simply a transpose. So, for most of our cases,
\begin{equation*} C = \frac{1}{n-1}B^{T}B\end{equation*}

In [4]:
def get_covariance_matrix(x):
    n = x.shape[0]
    assert n>=1
    
    return x.T.mm(x)/(n-1)

##### 3. Get eigen vectors and eigen values:

We get corresponding eigen vectors $V$ and eigen values $D$ from $C$. Since we did $B^{T}B$, the resultant matrix $C$ was symmetric so the eigen values should be real and the eigen vectors orthogonal

In [5]:
def eigen_vectors_and_values(x):
    e,v = torch.symeig(x, eigenvectors=True)
    
    # the above eigen values are in descending order and we prefer them in descending order
    sorted_e, indices = torch.sort(e,dim=0,descending=True)
    sorted_v = v.index_select(dim=0,index=indices)
    
    return sorted_e,sorted_v

##### 4. Select a subset of eigen vectors using a data ratio:

Its time to reduce the dimensions, given a user defined ratio default 90%, we select filters that contain as much information as the ratio from the original data. The cumulative energy content g for the jth eigenvector is the sum of the energy content across all of the eigenvalues from 1 through j after sorting them in descending order.

In [6]:
def get_number_of_features(eigen_values,ratio=0.9):
    sums = []
    a = 0
    for i in eigen_values:
        a += i.item()
        if a <0 :
            print("recieved a non-negative eigen value, setting it as 0")
            a = 0
        sums.append(a)
    
    # shortlist number of features using ratio
    g_p = sums[-1]
    l = len(sums) - 1
    while(True):
        l -= 1
        if (float(sums[l])/g_p)<ratio and l>=0:
            break
    l += 2 # because l here refers to index values so final features should be index values + 1 and another +1 because
    # the last l value failed
    
    return l  

def transform_data(x,eigen_vectors,features):
    return x.mm(eigen_vectors[:features].T)

### Applying PCA on MNIST

In [7]:
# Loading data
import gzip,os,pickle,math

def get_data(file_path, file_name):
    with gzip.open(os.path.join(file_path, file_name), 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

    return x_train, y_train, x_valid, y_valid


dir_ = "/home/akashe/PycharmProjects/data/MNIST"
filename = "mnist.pkl.gz"
train_x, train_y, test_x, test_y = get_data(dir_,filename)

In [8]:
type(train_x)

numpy.ndarray

In [9]:
# For now, we will work only with train_x
train_x_ = torch.as_tensor(train_x)
train_x.shape

(50000, 784)

In [10]:
# get column_wise mean
mean = columnwise_mean(train_x_)

In [11]:
# shift to zero mean with scaling
b = shift_mean(train_x_,mean,scale=False)

In [12]:
# get covariance matrix of b
c = get_covariance_matrix(b)

In [13]:
# get eigen vectors and values for c in descending order
e,v = eigen_vectors_and_values(c)

In [14]:
# get total features for which 80% of information is there
ratio = 0.9
l = get_number_of_features(e,ratio)

In [15]:
print(f'Number of features for {ratio*100}% of the information {l}')

Number of features for 90.0% of the information 235


In [16]:
no_of_desired_features = l
transformed_data = transform_data(b,v,no_of_desired_features)
transformed_data.shape

torch.Size([50000, 235])

#### Comparing results from sklearn

Note: sklearn approaches don't scale the inputs after shifting to zero mean

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [18]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(train_x)
pca = PCA(n_components=0.90)
Xt = pca.fit_transform(X_scaled)

In [19]:
sklearn_pca_features=len(Xt[0])
sklearn_pca_features

235

In [20]:
print(f'Getting {l} features from my implementation vs {sklearn_pca_features} from sklearn')

Getting 235 features from my implementation vs 235 from sklearn


#### Sources:

1. https://en.wikipedia.org/wiki/Principal_component_analysis
2. https://www.youtube.com/watch?v=fkf4IBRSeEc
3. In detail lecture: https://www.youtube.com/watch?v=WW3ZJHPwvyg